In [1]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None) ### This line makes all the columns display, rather than ellipses shorten

atp_2022_frame = pd.read_csv('C:/Users/aaron/Code/Data/atp_matches_2022.csv', parse_dates=[5])

atp_2023_frame = pd.read_csv('C:/Users/aaron/Code/Data/atp_matches_2023.csv', parse_dates=[5])

In [2]:
atp_2022_frame.loc[105:110]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
105,2022-9665,Melbourne,Hard,32,A,2022-01-03,273,105967,NaN,LL,Henri Laaksonen,R,185.0,SUI,29.7,105332,5.0,NaN,Benoit Paire,R,196.0,FRA,32.6,4-6 6-3 5-2 RET,3,R32,118.0,5.0,1.0,80.0,62.0,43.0,9.0,13.0,8.0,11.0,3.0,8.0,107.0,40.0,31.0,28.0,13.0,15.0,20.0,98.0,765.0,46.0,1245.0
106,2022-9665,Melbourne,Hard,32,A,2022-01-03,272,144719,NaN,NaN,Jaume Munar,R,183.0,ESP,24.6,104731,NaN,NaN,Kevin Anderson,R,203.0,RSA,35.6,6-4 6-4,3,R32,101.0,1.0,4.0,67.0,43.0,34.0,11.0,10.0,1.0,1.0,9.0,3.0,73.0,46.0,28.0,15.0,10.0,7.0,9.0,77.0,848.0,80.0,829.0
107,2022-9665,Melbourne,Hard,32,A,2022-01-03,271,202195,NaN,Q,Maxime Cressy,R,198.0,USA,24.6,208014,NaN,Q,Rinky Hijikata,U,NaN,AUS,20.8,6-3 6-0,3,R32,63.0,9.0,2.0,45.0,33.0,24.0,10.0,8.0,0.0,0.0,1.0,4.0,43.0,23.0,12.0,8.0,7.0,2.0,6.0,112.0,683.0,375.0,113.0
108,2022-9667,Adelaide 2,Hard,32,A,2022-01-10,300,106423,NaN,WC,Thanasi Kokkinakis,R,196.0,AUS,25.7,126239,NaN,NaN,Arthur Rinderknech,R,196.0,FRA,26.4,6-7(6) 7-6(5) 6-3,3,F,158.0,18.0,0.0,92.0,71.0,64.0,11.0,16.0,0.0,0.0,15.0,0.0,104.0,68.0,55.0,19.0,17.0,1.0,3.0,145.0,480.0,58.0,1057.0
109,2022-9667,Adelaide 2,Hard,32,A,2022-01-10,299,126239,NaN,NaN,Arthur Rinderknech,R,196.0,FRA,26.4,144895,NaN,Q,Corentin Moutet,L,178.0,FRA,22.7,6-1 6-3,3,SF,76.0,6.0,2.0,46.0,30.0,27.0,8.0,8.0,2.0,2.0,0.0,0.0,46.0,33.0,18.0,4.0,8.0,2.0,6.0,58.0,1057.0,115.0,643.0
110,2022-9667,Adelaide 2,Hard,32,A,2022-01-10,298,106423,NaN,WC,Thanasi Kokkinakis,R,196.0,AUS,25.7,105227,4.0,NaN,Marin Cilic,R,198.0,CRO,33.2,6-2 3-6 7-6(10),3,SF,158.0,10.0,1.0,93.0,65.0,49.0,14.0,14.0,5.0,7.0,7.0,3.0,99.0,58.0,42.0,22.0,15.0,4.0,7.0,145.0,480.0,29.0,1760.0


In [3]:
atp_2022_frame['surface_cat'] = atp_2022_frame['surface'].astype("category").cat.codes ## Add a surface category column
atp_2023_frame['surface_cat'] = atp_2023_frame['surface'].astype("category").cat.codes ## Add a surface category column

atp_2022_frame.dropna(subset = ['winner_ht', 'loser_ht'], inplace = True) ## Drop all null height values
atp_2023_frame.dropna(subset = ['winner_ht', 'loser_ht'], inplace = True) ##Drop all null height values

atp_2022_frame['winner_rank'].fillna(1000, inplace = True)
atp_2023_frame['loser_rank'].fillna(1000, inplace = True)

In [4]:
##### figure out a good way to make a dataframe for just one player. Change it all from winner and loser to player and opponent.

def createPlayerFrame(playerId, year):

    if (year == '2022'):
        frameName = atp_2022_frame
    elif (year == '2023'):
        frameName = atp_2023_frame
    else:
        return ('Bad year input, must be 2022 or 2023 represented as a String')


    

    columns_to_keep = ['tourney_date', 'winner_rank', 'loser_rank', 'winner_age', 'loser_age', 'winner_ht', 'loser_ht', 'surface_cat', 'winner_hand', 'loser_hand']
    
    winningRows = frameName[frameName['winner_id'] == playerId] ## get all wins for player
    losingRows = frameName[frameName['loser_id'] == playerId] ## get all losses for player

    losingRows = losingRows[columns_to_keep] ## get rid of most columns
    winningRows = winningRows[columns_to_keep] ## get rid of most columns

    winning_rename_lib = {'winner_rank':'player_rank', 'loser_rank':'opponent_rank', 'winner_age':'player_age', 'loser_age':'opponent_age', 'winner_ht':'player_height', 'loser_ht':'opponent_height', 'winner_hand':'player_hand', 'loser_hand':'opponent_hand'} #rename columns
    losing_rename_lib = {'loser_rank':'player_rank', 'winner_rank':'opponent_rank', 'loser_age':'player_age', 'winner_age':'opponent_age', 'loser_ht':'player_height', 'winner_ht':'opponent_height', 'loser_hand':'player_hand', 'winner_hand':'opponent_hand'} #rename columns
    
    winningRows.rename(columns = winning_rename_lib, inplace = True) #rename a bunch
    losingRows.rename(columns = losing_rename_lib, inplace = True) #rename a bunch
    winningRows['result'] = 'W'
    losingRows['result'] = 'L'
    
    allPlayerMatchRows = pd.concat([winningRows, losingRows])

    allPlayerMatchRows['player_hand'] = allPlayerMatchRows['player_hand'].astype("category").cat.codes 
    allPlayerMatchRows['opponent_hand'] = allPlayerMatchRows['opponent_hand'].astype("category").cat.codes
    
    #playerFrame = allPlayerMatchRows[columns_to_keep]
    
    #playerFrame.rename(columns = rename_lib, inplace = True)

    return (allPlayerMatchRows)

In [5]:
playerId = 144895
playerFrame2022 = createPlayerFrame(playerId, '2022')
playerFrame2023 = createPlayerFrame(playerId, '2023')

playerFrame2022

,tourney_date,player_rank,opponent_rank,player_age,opponent_age,player_height,opponent_height,surface_cat,player_hand,opponent_hand,result
76,2022-01-03,92.0,103.0,22.7,18.6,178.0,185.0,2,0,1,W
111,2022-01-10,115.0,83.0,22.7,27.6,178.0,183.0,2,0,0,W
116,2022-01-10,115.0,38.0,22.7,29.9,178.0,188.0,2,0,1,W
124,2022-01-10,115.0,50.0,22.7,31.7,178.0,193.0,2,0,1,W
171,2022-01-17,100.0,160.0,22.7,27.8,178.0,185.0,2,0,1,W
1261,2022-05-23,139.0,264.0,23.0,37.1,178.0,183.0,0,0,1,W
1854,2022-07-25,116.0,65.0,23.2,23.8,178.0,191.0,0,0,1,W
2121,2022-08-29,112.0,295.0,23.3,37.4,178.0,183.0,2,0,1,W
2172,2022-08-29,112.0,22.0,23.3,26.9,178.0,188.0,2,0,1,W
2197,2022-08-29,112.0,66.0,23.3,27.3,178.0,185.0,2,0,1,W


In [6]:
random_forest = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1, verbose = 1)

In [7]:
predictors = ["player_rank", "opponent_rank", "player_age", "opponent_age", "player_height", "opponent_height", "surface_cat", "player_hand", "opponent_hand"]
less_predictors = ["player_rank", "opponent_rank"]

In [8]:
random_forest.fit(playerFrame2022[predictors], playerFrame2022['result'])

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1,
                       verbose=1)

In [9]:
#pd.set_option('display.max_rows', None) ## use this line if you wanna see all rows
#playerFrame2023[predictors]

In [10]:
predictions = random_forest.predict(playerFrame2023[predictors])

predictions

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


array(['L', 'W', 'W', 'W', 'W', 'L', 'W', 'W', 'W', 'L', 'W', 'W', 'W',
       'W', 'W', 'L', 'W', 'L', 'W', 'L', 'W'], dtype=object)

In [11]:
accuracy = accuracy_score(playerFrame2023['result'], predictions)

accuracy

0.42857142857142855

In [12]:
dummyData = {'tourney_date':['2022-08-17'],
            'player_rank':['18.0'],
            'opponent_rank':['11.0'],
            'player_age':['28.2'],
            'opponent_age':['33.5'],
            'result':['L']}

testFrame = pd.DataFrame(dummyData)

testFrame

#testFrame = pd.DataFrame().reindex_like(shapoFrame).dropna()
#testFrame.concat(dummyData, ignore_index=True)

#testFrame
#random_forest.predict(testFrame[predictors])

#shapoFrame[predictors]

,tourney_date,player_rank,opponent_rank,player_age,opponent_age,result
0,2022-08-17,18.0,11.0,28.2,33.5,L
